## Transformer for Kwere

In [1]:
import string


In [2]:
VOCAB_SIZE = 48
ALPHABET = [i for i in string.ascii_lowercase +string.digits + r'!"(),-.:;?'+ "'"]
#[ !"'(),-.0-9:;?a-z]


In [3]:
from architecture import BPETokenizer
kwere_path = ["./train-04/cwe-train.txt"]
tokenizer = BPETokenizer(VOCAB_SIZE, ALPHABET)  # same tokenizers for both lang
tokenizer.bpe_train(kwere_path)
tokenizer.save_tokenizer("pretrained_token") #same tokenizers for both lang


2022-10-27 20:47:04.463130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import tensorflow as tf
from transformers import GPT2Config, TFGPT2Model, GPT2Tokenizer
#transfer to gpt2tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("pretrained_token",
                                          pad_token="<pad>",
                                          mask_token="<mask>", 
                                          max_length=128,
                                          is_split_into_words=False)

config = GPT2Config(
    vocab_size=tokenizer.vocab_size,
    n_embd=256,
    n_layer=4,
    n_head = 4
)


/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
# creating the TFGPT2
model = TFGPT2Model(config)
model(model.dummy_inputs)
model.summary()


2022-10-27 20:47:10.349373: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "tfgpt2_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 3434240   
 r)                                                              
                                                                 
Total params: 3,434,240
Trainable params: 3,434,240
Non-trainable params: 0
_________________________________________________________________


**Language Model for Kwere**

In [6]:
kwere_corpus = open(kwere_path[0]).read()
kwere_tokenized = tokenizer.encode(kwere_corpus)


In [7]:
from architecture import get_tensor_data
kwere_dataset = get_tensor_data(kwere_tokenized,5,28)

In [8]:
# defining our optimizer
optimizer = tf.keras.optimizers.Adam(
    learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
# definining our loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# compiling the model
model.compile(optimizer=optimizer, loss=[
              loss, *[None] * model.config.n_layer])


In [9]:
num_epoch = 5
history = model.fit(kwere_dataset, epochs=num_epoch)

Epoch 1/10
   449/120686 [..............................] - ETA: 32:49 - loss: 3.1356

In [ ]:
#model.save_pretrained('kwere_lm')
tokenizer.save_pretrained("kwere_tok")

('kwere_tok/tokenizer_config.json',
 'kwere_tok/special_tokens_map.json',
 'kwere_tok/vocab.json',
 'kwere_tok/merges.txt',
 'kwere_tok/added_tokens.json')

**Language Model for Swahili**

In [16]:
sw_corpus = open("./train-04/sw-train.txt").read()
sw_tokenized = tokenizer.encode(sw_corpus)
sw_dataset = get_tensor_data(sw_tokenized, 50)
model.fit(sw_dataset, epochs=2)


Epoch 1/2
39853/65435 [=================>............] - ETA: 2:08:40 - loss: 1.7429

KeyboardInterrupt: 

In [ ]:
model.save_pretrained('sw_lm')
